## CSV to Zarr

#### Import libraries

In [2]:
import os
import fsspec
import numpy as np
import pandas as pd
import time

#### Generate numpy array

In [9]:
fs = fsspec.filesystem('s3', anon=True)
all_files = fs.glob(f's3://imos-data-pixeldrill/viet-test/csv/*.csv')

start_time = time.time()
nps = []
headers = []
for i, file in enumerate(all_files):
    s3_fn = 's3://' + file
    with fs.open(s3_fn, "rb") as f:
        df_metadata = pd.read_csv(f, nrows=1)
    with fs.open(s3_fn, "rb") as f:
        df_data = pd.read_csv(f, skiprows=[0,1])

    df = pd.concat([df_metadata, df_data], axis=0, ignore_index=True)
    df.ffill(axis=0, inplace=True)
    headers = df.columns #same columns across the iterations
    np_array = df.to_numpy()
    nps.append(np_array)
    
print("---------- Total: %.2f seconds ----------" % (time.time() - start_time))

---------- Total: 4.71 seconds ----------


In [5]:
# # check if columns across the CSV files are identical
# all(x==all_cols[0] for x in all_cols)